# Assignment 1: Web Scraping

## Objective

Data scientists often need to crawl data from websites and turn the crawled data (HTML pages) to structured data (tables). Thus, web scraping is an essential skill that every data scientist should master. In this assignment, you will learn the followings:


* How to use [requests](http://www.python-requests.org/en/master/) to download HTML pages from a website?
* How to select content on a webpage with [lxml](http://lxml.de/)? 

You can either use Spark DataFrame or [pandas.DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) to do the assignment. In comparison, pandas.DataFrame has richer APIs, but is not good at distributed computing.

## Preliminary

If this is your first time to write a web scraper, you need to learn some basic knowledge of HTML, DOM, and XPath. I found that this is a good resource: [https://data-lessons.github.io](https://data-lessons.github.io/library-webscraping-DEPRECATED/). Please take a look at

* [Selecting content on a web page with XPath
](https://data-lessons.github.io/library-webscraping-DEPRECATED/xpath/)
* [Web scraping using Python: requests and lxml](https://data-lessons.github.io/library-webscraping-DEPRECATED/04-lxml/). 

Please let me know if you find a better resource. I'll share it with the other students.

## Overview

Imagine you are a data scientist working at SFU. One day, you want to analyze CS faculty data and answer two interesting questions:

1. Who are the CS faculty members?
2. What are their research interests?

To do so, the first thing is to figure out what data to collect.

## Task 1: SFU CS Faculty Members

You find that there is a web page in the CS school website, which lists all the faculty members as well as their basic information. 

In Task 1, your job is to write a web scraper to extract the faculty information from this page: [https://www.sfu.ca/computing/people/faculty.html](https://www.sfu.ca/computing/people/faculty.html).




### (a) Crawling Web Page

A web page is essentially a file stored in a remote machine (called web server). You can use [requests](http://www.python-requests.org/en/master/) to open such a file and read data from it. Please complete the following code to download the HTML page and save it as a text file (like [this](./faculty.txt)). 

In [1]:
import requests


# 1. Download the webpage
url = 'https://www.sfu.ca/computing/people/faculty.html'
page = requests.get( url )
html = page.content

# 2. Save it as a text file (named faculty.txt)
open('faculty.txt','wb').write(html)

125036

### (b) Extracting Structured Data

An HTML page follows the Document Object Model (DOM). It models an HTML page as a tree structure wherein each node is an object representing a part of the page. The nodes can be searched and extracted programmatically using XPath. Please complete the following code to transform the above HTML page to a CSV file (like [this](./faculty_table.csv)). 

In [2]:
import lxml.html as lh

# 1. Open faculty.txt
html = open('faculty.txt','r').read()

# 2. Parse the HTML page as a tree structure
tree = lh.fromstring(html)

# 3. Extract related content from the tree using XPath
prof_container = tree.xpath('//div[@class="textimage section"]')

name = []
rank = []
area = []
profile = []
homepage = []

for container in prof_container:
    prof = container.xpath('./div/div[@class="text"]')
    text = prof[0].xpath('./h4/text()')[0].split(", ")
    
    name.append(text[0])
    
    rank.append(text[1].split('\n')[0].strip('\xa0'))
    
    if(text[0] != "Fred Popowich"):
        area.append(prof[0].xpath("./p/text()")[0].strip(': ').strip('\xa0'))
    else:
        area.append(prof[0].xpath("./h4/text()")[-1][1:])
    
    if len(prof[0].xpath('./p[2]')) is 0:
        profile.append('http://www.sfu.ca'+prof[0].xpath('./p/a/@href')[0])
        homepage.append(prof[0].xpath('./p/a[2]/@href')[0])
    elif len(prof[0].xpath('./p[2]/a')) is 0:
        profile.append('http://www.sfu.ca'+prof[0].xpath('./p[3]/a/@href')[0])
        homepage.append(prof[0].xpath('./p[3]/a[2]/@href')[0])
    else:
        profile.append('http://www.sfu.ca'+prof[0].xpath('./p[2]/a/@href')[0].replace('http://www.sfu.ca',''))
        if len(prof[0].xpath('./p[2]/a[2]')) is 0:
            homepage.append('')
        else: 
            homepage.append(prof[0].xpath('./p[2]/a[2]/@href')[0])

    
# 4. Save the extracted content as an csv file (named faculty_table.csv)
import pandas as pd

df = pd.DataFrame({'name': name,
                  'rank': rank,
                  'area': area,
                  'profile': profile,
                  'homepage': homepage})
df.to_csv('faculty_table.csv',index=False)


## Task 2: Research Interests

Suppose you want to know the research interests of each faculty. However, the above crawled web page does not contain such information. 

### (a) Crawling Web Page

You notice that such information can be found on the profile page of each faculty. For example, you can find the research interests of Dr. Jiannan Wang from [http://www.sfu.ca/computing/people/faculty/jiannanwang.html](http://www.sfu.ca/computing/people/faculty/jiannanwang.html). 


Please complete the following code to download the profile pages and save them as text files. There are 60 faculties, so you need to download 60 web pages in total. 

In [5]:
import requests
import pandas as pd

# 1. Download the profile pages of 60 faculties
df = pd.read_csv('faculty_table.csv')
name = df['name'].tolist()
profile = df['profile'].tolist()
# 2. Save each page as a text file
profiles = []

for i in range(len(profile)):
    page = requests.get(profile[i])
    html = page.content
    file = name[i].replace(' ','_')+"_profile.txt"
    open(file,"wb").write(html)
    profiles.append(file)
    

### (b) Extracting Structured Data

Please complete the following code to extract the research interests of each faculty, and generate a file like [this](./faculty_more_table.csv). 

In [6]:
import lxml.html as lh
import os
import numpy as np

# 1. Open each text file and parse it as a tree structure 
# 2. Extract the research interests from each tree using XPath
# 3. Add the extracted content to faculty_table.csv
# 4. Generate a new CSV file, named faculty_more_table.csv

research_interests = []

for filename in profiles:
    html = open(filename,'r',encoding="utf-8").read()
    tree = lh.fromstring(html)
    research = tree.xpath("//div[@class='parsys_column cq-colctrl-lt0 ']")
    ri = []
    li = []
    stri = ["Research interests","Research Interests"]
    locs = ['./div[2]/div[2]/div/h2/text()','./div[1]/div[4]/div/h2/text()','./div[2]/div[3]/div/h2/text()','./div[2]/div[4]/div/h2/text()']

    for path in locs:
        if len(research[0].xpath(path)) != 0 and research[0].xpath(path)[0] in stri:
            li += research[0].xpath(path[:-10])[0].xpath('./ul//li')
            for i in li:
                ri.append(i.text_content())
        else:
            ri = ri
            
    if '\n' in ri:    
        ri = list(filter(lambda a: a != '\n', ri))
        
    for i in range(len(ri)):
        ri[i] = ri[i].replace('\xa0','').replace('\n','')
    
    research_interests.append(ri)

res_int = np.asarray(research_interests)
df["research_interests"]=res_int
df.to_csv('faculty_more_table.csv',index=False)

## Submission

Complete the code in this [notebook](A1.ipynb), and submit it to the CourSys activity `Assignment 1`.